# Развёртка vLLM и локальное взаимодействие

In [1]:
install = False
if install:
  %pip install -q vllm
  %pip install -q openai
  %pip install -q mlflow

In [2]:
import subprocess
import time
import threading
import requests
import json
import openai
import mlflow
import pandas as pd

## 1. Развернуть vLLM с подходящий моделью

In [3]:
from google.colab import userdata
from huggingface_hub import login

hf_access_token=userdata.get('HF_token')
login(token=hf_access_token)

In [4]:
# --- ШАГ 1: Запуск vLLM ---
print("--- ЗАПУСК vLLM SERVER ---")
vllm_cmd = [
    "python", "-m", "vllm.entrypoints.openai.api_server",
    "--model", "Qwen/Qwen2.5-0.5B-Instruct",
    "--host", "0.0.0.0",
    "--port", "8000",
    "--gpu-memory-utilization", "0.6",
    "--max-num-seqs", "8",
]

vllm_process = subprocess.Popen(
    vllm_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True
)

def log_vllm_output(pipe, prefix=""):
    for line in iter(pipe.readline, ""):
        print(f"[vLLM] {prefix}{line.strip()}")
    pipe.close()

vllm_stdout_thread = threading.Thread(target=log_vllm_output, args=(vllm_process.stdout, "OUT: "))
vllm_stderr_thread = threading.Thread(target=log_vllm_output, args=(vllm_process.stderr, "ERR: "))
vllm_stdout_thread.start()
vllm_stderr_thread.start()

print("Ожидание запуска vLLM API сервера (ожидается 'Application startup complete.')...")
time.sleep(200) # Подождем, пока vLLM полностью не запустится
print("--- vLLM SERVER ЗАПУЩЕН (предположительно) ---")

# --- ШАГ 2: Установка и запуск localtunnel ---
print("\n--- УСТАНОВКА И ЗАПУСК LOCALTUNNEL ---")
try:
    subprocess.run(["npm", "install", "-g", "localtunnel"], check=True)
except subprocess.CalledProcessError:
    print("Ошибка установки localtunnel. Проверьте npm.")
    exit()

lt_cmd = ["lt", "--port", "8000"]
lt_process = subprocess.Popen(lt_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

def log_lt_output(pipe, prefix=""):
    for line in iter(pipe.readline, ""):
        print(f"[LT] {prefix}{line.strip()}")
        # Пытаемся найти URL в выводе stderr
        if "your url is:" in line:
            global found_url
            found_url = line.split("your url is:")[1].strip()
            print(f"\n### TUNNEL URL НАЙДЕН: {found_url} ###")
            print("### КОПИРУЕМ ЭТОТ URL ДЛЯ СЛЕДУЮЩЕГО ШАГА ###")
    pipe.close()

global found_url
found_url = None

lt_stdout_thread = threading.Thread(target=log_lt_output, args=(lt_process.stdout, "OUT: "))
lt_stderr_thread = threading.Thread(target=log_lt_output, args=(lt_process.stderr, "ERR: "))
lt_stdout_thread.start()
lt_stderr_thread.start()

print("Ожидание создания туннеля localtunnel...")
# Ждем несколько секунд, чтобы localtunnel успел создать соединение и вывести URL
time.sleep(20)

if found_url:
    print(f"\n*** ГОТОВО ***")
    print(f"Наш API URL: {found_url}/v1")
    print("Копируем его и используем в следующей ячейке.")
else:
    print("\n--- Предупреждение ---")
    print("URL туннеля не был найден автоматически в выводе.")
    print("Проверить вывод ячейки выше на наличие строки 'your url is: ...'")
    print("Если URL не появился, возможно, localtunnel завис или не смог подключиться.")

# Теперь процесс vLLM и localtunnel работают в фоне.
# Ячейка завершена, но процессы остаются активными.

--- ЗАПУСК vLLM SERVER ---
Ожидание запуска vLLM API сервера (ожидается 'Application startup complete.')...
[vLLM] ERR: 2026-01-26 18:07:16.480477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
[vLLM] ERR: WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
[vLLM] ERR: E0000 00:00:1769450836.523656    4756 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
[vLLM] ERR: E0000 00:00:1769450836.535383    4756 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[vLLM] ERR: W0000 00:00:1769450836.562670    4756 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
[vLLM] 

## 2. Взаимодействие с локальной моделью

- Используя библиотеку для http запросов (httpx / requests / etc) обратитесь к модели и получите ответ (например с вопросом "What is the capital of Germany?").
- Сделайте тоже самое используя библиотеку openai (тут придется столкнуться с совместимостью OpenAI API и разных моделей, будьте внимательны).


### Библиотека для http запросов

In [5]:
# --- ВСТАВИМ СВОЙ URL ОТ LOCALTUNNEL (без /v1 на конце) ---
OUR_LT_URL = "https://tired-mammals-poke.loca.lt" # <- НАШ РЕАЛЬНЫЙ URL ОТ LOCALTUNNEL

# Формируем правильный API_URL, добавляя /v1
API_URL = f"{OUR_LT_URL.rstrip('/')}/v1" # rstrip убирает лишний / на конце, если он есть

print(f"Используем API_URL: {API_URL}")

url = f"{API_URL}/chat/completions" # Теперь это будет OUR_LT_URL/v1/chat/completions

headers = {
    "Content-Type": "application/json"
}

data = {
    "model": "Qwen/Qwen2.5-0.5B-Instruct",
    "messages": [
        {"role": "user", "content": "What is the capital of Germany?"}
    ],
    "temperature": 0.7,
    "max_tokens": 150
}

try:
    print(f"Отправляю запрос к {url}")
    response = requests.post(url, headers=headers, data=json.dumps(data))
    response.raise_for_status()

    result = response.json()
    # print(json.dumps(result, indent=2)) # Для просмотра полного ответа

    answer = result['choices'][0]['message']['content'].strip()
    print(f"\n--- Ответ модели ---\n{answer}\n-------------------")

except requests.exceptions.RequestException as e:
    print(f"Произошла ошибка при запросе: {e}")
    if response is not None:
        print(f"Статус код: {response.status_code}")
        print(f"Тело ответа: {response.text}")
except KeyError as e:
    print(f"Ошибка при парсинге ответа: {e}")
    print(f"Полученный JSON: {result}")

Используем API_URL: https://tired-mammals-poke.loca.lt/v1
Отправляю запрос к https://tired-mammals-poke.loca.lt/v1/chat/completions
[vLLM] OUT: (APIServer pid=4756) INFO:     35.240.191.247:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK

--- Ответ модели ---
The capital of Germany is Berlin.
-------------------


### Библиотека openai

In [6]:
# Настраиваем клиент OpenAI для работы с vLLM
client = openai.OpenAI(
    base_url=f"{OUR_LT_URL.rstrip('/')}/v1",
    api_key="not-needed"  # Для vLLM API ключ не требуется
)

# Создаем запрос
try:
    print(f"Отправляю запрос к {OUR_LT_URL}/v1")

    response = client.chat.completions.create(
        model="Qwen/Qwen2.5-0.5B-Instruct",
        messages=[
            {"role": "user", "content": "What is the capital of Germany?"}
        ],
        temperature=0.7,
        max_tokens=150
    )

    print(f"\n--- Ответ модели ---")
    print(response.choices[0].message.content)
    print(f"-------------------")

    # Дополнительная информация о запросе (опционально)
    print(f"\nДополнительная информация:")
    print(f"Модель: {response.model}")
    print(f"Использовано токенов: {response.usage.total_tokens}")

except openai.APIError as e:
    print(f"Ошибка API: {e}")
except Exception as e:
    print(f"Произошла ошибка: {e}")

Отправляю запрос к https://tired-mammals-poke.loca.lt/v1
[vLLM] OUT: (APIServer pid=4756) INFO:     35.240.191.247:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK

--- Ответ модели ---
The capital of Germany is Berlin.
-------------------

Дополнительная информация:
Модель: Qwen/Qwen2.5-0.5B-Instruct
Использовано токенов: 44


## 3. Mlflow и нтеграция кастомной genai метрики:

- Создать кастомную метрику для оценки эксперимента используя локальную модель.
- Прогнать эксперимент на любом датафрейме с использованием локальной модели для расчета метрики.

In [7]:
# Укажите путь к вашему хранилищу
mlflow.set_tracking_uri("file:///content/mlruns")

# Получить информацию о последнем запуске в эксперименте
experiment_name = "Real_Scenario_Local_LLMEval"
mlflow.set_experiment(experiment_name)

/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/01/26 18:12:29 INFO mlflow.tracking.fluent: Experiment with name 'Real_Scenario_Local_LLMEval' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/235368573044722462', creation_time=1769451149541, experiment_id='235368573044722462', last_update_time=1769451149541, lifecycle_stage='active', name='Real_Scenario_Local_LLMEval', tags={}>

In [8]:
# --- НАСТРОЙКИ ---
OUR_API_BASE_URL = OUR_LT_URL # <- НАШ РЕАЛЬНЫЙ URL ОТ ТУННЕЛЯ (БЕЗ /v1)
MODEL_NAME_FOR_EVAL = "Qwen/Qwen2.5-0.5B-Instruct"
# -----------------

def generate_answer(question, context=None):
    """
    Генерирует ответ на вопрос с помощью локальной LLM (через API).
    """
    api_url = f"{OUR_API_BASE_URL.rstrip('/')}/v1/chat/completions"

    context_str = context if context else ""
    prompt = f"{context_str}\n\nQuestion: {question}\nAnswer:"

    payload = {
        "model": MODEL_NAME_FOR_EVAL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.1, # Для более детерминированного ответа
        "max_tokens": 100,
    }

    try:
        response = requests.post(api_url, headers={"Content-Type": "application/json"}, json=payload)
        response.raise_for_status()
        result = response.json()
        answer = result['choices'][0]['message']['content'].strip()
        return answer
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при генерации для вопроса '{question[:30]}...': {e}")
        return "Error generating answer"
    except Exception as e:
        print(f"Неожиданная ошибка при генерации для вопроса '{question[:30]}...': {e}")
        return "Error generating answer"


In [9]:
def evaluate_prediction_quality(predictions, contexts=None, references=None):
    """
    Оценивает качество предсказаний с помощью локальной LLM.
    """
    scores = []
    api_url = f"{OUR_API_BASE_URL.rstrip('/')}/v1/chat/completions"

    for i, pred in enumerate(predictions):
        context_str = contexts[i] if contexts else "Контекст не предоставлен."
        reference_str = references[i] if references else "Эталонный ответ не предоставлен."

        evaluation_prompt = f"""
        Задание: Оцените качество следующего предсказанного ответа по шкале от 0 до 10.
        Контекст: {context_str}
        Вопрос: (можно вывести из контекста или подразумевается)
        Предсказанный ответ: {pred}
        Эталонный ответ: {reference_str}

        Обоснуйте оценку кратко, а затем поставьте оценку в формате "Оценка: X", где X - число от 0 до 10.
        """

        payload = {
            "model": MODEL_NAME_FOR_EVAL,
            "messages": [{"role": "user", "content": evaluation_prompt}],
            "temperature": 0.0, # Для детерминированности
            "max_tokens": 100,
        }

        try:
            response = requests.post(api_url, headers={"Content-Type": "application/json"}, json=payload)
            response.raise_for_status()
            result = response.json()
            full_response_text = result['choices'][0]['message']['content'].strip()

            import re
            match = re.search(r'Оценка:\s*(\d+\.?\d*)', full_response_text)
            if match:
                score = float(match.group(1))
                score = max(0.0, min(10.0, score))
            else:
                print(f"Предупреждение: Не удалось извлечь оценку из ответа для предсказания {i}: {full_response_text}")
                score = 0.0

        except requests.exceptions.RequestException as e:
            print(f"Ошибка при оценке для предсказания {i}: {e}")
            score = 0.0
        except Exception as e:
            print(f"Неожиданная ошибка при оценке {i}: {e}")
            score = 0.0

        scores.append(score)

    return scores


In [10]:
# --- Подготовка тестовых данных ---
questions = [
    "What is the capital of France?",
    "Who wrote 'Romeo and Juliet'?",
    "What is the boiling point of water?",
    "How many planets are in our solar system?",
    "What is the largest mammal?",
    "What is the chemical symbol for gold?",
    "Who painted the Mona Lisa?",
    "What is the tallest mountain in the world?",
    "What is the currency of Japan?",
    "What is the fastest land animal?",
]

contexts = [
    "Simple geography question.",
    "Simple literature question.",
    "Simple science question.",
    "Simple astronomy question.",
    "Simple biology question.",
    "Simple chemistry question.",
    "Simple art question.",
    "Simple geography question.",
    "Simple economics question.",
    "Simple biology question.",
]

# Истинные (эталонные) ответы
true_references = [
    "Paris",
    "William Shakespeare",
    "100°C at sea level",
    "Eight",
    "The blue whale is the largest mammal.",
    "Au",
    "Leonardo da Vinci",
    "Mount Everest",
    "Yen",
    "The cheetah is the fastest land animal.",
]

# --- Генерация предсказаний основной моделью ---
print("Начинаю генерацию предсказаний основной моделью...")
generated_predictions = []
for q in questions:
    ans = generate_answer(q)
    generated_predictions.append(ans)
    print(f"Вопрос: {q}")
    print(f"Предсказание: {ans}\n---")

# --- Подготовка данных для оценки ---
df_for_evaluation = pd.DataFrame({
    "question": questions,
    "context": contexts,
    "prediction": generated_predictions, # <-- Сгенерировано моделью
    "reference": true_references,       # <-- Истина
})

print("\n--- Подготовленные данные для оценки ---")
print(df_for_evaluation[['question', 'prediction', 'reference']].head()) # Показываем первые 5

# --- Запуск MLflow эксперимента ---
experiment_name = "Real_Scenario_Local_LLMEval"
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    RUN_ID = run.info.run_id
    # Логирование параметров
    mlflow.log_param("model_for_generation", MODEL_NAME_FOR_EVAL)
    mlflow.log_param("model_for_evaluation", MODEL_NAME_FOR_EVAL)
    mlflow.log_param("evaluation_api_base", OUR_API_BASE_URL)
    mlflow.log_param("num_samples", len(questions))

    # Вычисление кастомной метрики
    print("\nНачинаю вычисление кастомной метрики...")
    eval_scores = evaluate_prediction_quality(
        df_for_evaluation['prediction'].tolist(),
        df_for_evaluation['context'].tolist(), # contexts могут быть полезны для оценки
        df_for_evaluation['reference'].tolist()
    )

    mean_eval_score = sum(eval_scores) / len(eval_scores) if eval_scores else 0.0

    # Логирование результатов
    mlflow.log_metric("custom_llm_eval_score_mean", mean_eval_score)
    # Логирование индивидуальных оценок (опционально)
    for i, score in enumerate(eval_scores):
        mlflow.log_metric(f"custom_llm_eval_score_item_{i}", score)

    # Логирование данных (опционально, для анализа)
    # df_for_evaluation.to_csv("eval_data.csv", index=False)
    # mlflow.log_artifact("eval_data.csv")

    print(f"\nЭксперимент завершён.")
    print(f"Средняя оценка (кастомная метрика): {mean_eval_score:.2f}")
    print(f"Данные и результаты доступны в MLflow UI.")

print("\nПроверьте MLflow UI для просмотра полных результатов эксперимента.")

Начинаю генерацию предсказаний основной моделью...
[vLLM] OUT: (APIServer pid=4756) INFO:     35.240.191.247:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
Вопрос: What is the capital of France?
Предсказание: The capital of France is Paris.
---
[vLLM] OUT: (APIServer pid=4756) INFO:     35.240.191.247:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
Вопрос: Who wrote 'Romeo and Juliet'?
Предсказание: "Romeo and Juliet" was written by William Shakespeare. This play is one of the most famous tragedies in Western literature and has been performed countless times across different languages and cultures. The story revolves around two young lovers who fall in love with each other but are rejected by their families due to their gender differences. The play explores themes such as love, revenge, and the consequences of societal expectations on individuals. It has had a lasting impact on literature and continues to be studied and appreciated today.
---
[vLLM] OUT: (APIServer pid=4756) INFO 01-2

In [11]:
!ls -R /content/mlruns/

/content/mlruns/:
0  235368573044722462

/content/mlruns/0:
meta.yaml

/content/mlruns/235368573044722462:
2bc4c01f147d4e91a174a56c2d4b5acb  meta.yaml

/content/mlruns/235368573044722462/2bc4c01f147d4e91a174a56c2d4b5acb:
artifacts  meta.yaml  metrics  params  tags

/content/mlruns/235368573044722462/2bc4c01f147d4e91a174a56c2d4b5acb/artifacts:

/content/mlruns/235368573044722462/2bc4c01f147d4e91a174a56c2d4b5acb/metrics:
custom_llm_eval_score_item_0  custom_llm_eval_score_item_6
custom_llm_eval_score_item_1  custom_llm_eval_score_item_7
custom_llm_eval_score_item_2  custom_llm_eval_score_item_8
custom_llm_eval_score_item_3  custom_llm_eval_score_item_9
custom_llm_eval_score_item_4  custom_llm_eval_score_mean
custom_llm_eval_score_item_5

/content/mlruns/235368573044722462/2bc4c01f147d4e91a174a56c2d4b5acb/params:
evaluation_api_base  model_for_evaluation  model_for_generation  num_samples

/content/mlruns/235368573044722462/2bc4c01f147d4e91a174a56c2d4b5acb/tags:
mlflow.runName	mlflow.sour

In [12]:
EXPERIMENT_ID = mlflow.get_experiment_by_name(experiment_name).experiment_id
!cat /content/mlruns/{EXPERIMENT_ID}/{RUN_ID}/meta.yaml
!ls /content/mlruns/{EXPERIMENT_ID}/{RUN_ID}/metrics
!ls /content/mlruns/{EXPERIMENT_ID}/{RUN_ID}/params

artifact_uri: file:///content/mlruns/235368573044722462/2bc4c01f147d4e91a174a56c2d4b5acb/artifacts
end_time: 1769451204318
entry_point_name: ''
experiment_id: '235368573044722462'
lifecycle_stage: active
run_id: 2bc4c01f147d4e91a174a56c2d4b5acb
run_name: rebellious-horse-690
source_name: ''
source_type: 4
source_version: ''
start_time: 1769451190227
status: 3
tags: []
user_id: root
custom_llm_eval_score_item_0  custom_llm_eval_score_item_6
custom_llm_eval_score_item_1  custom_llm_eval_score_item_7
custom_llm_eval_score_item_2  custom_llm_eval_score_item_8
custom_llm_eval_score_item_3  custom_llm_eval_score_item_9
custom_llm_eval_score_item_4  custom_llm_eval_score_mean
custom_llm_eval_score_item_5
evaluation_api_base  model_for_evaluation  model_for_generation  num_samples
